In [1]:
import tkinter as tk
from tkinter import messagebox, filedialog
from PIL import Image, ImageTk
from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib import colors
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
import os
import numpy as np
import pandas as pd
import joblib
from pyedflib import highlevel


def extract_features(eeg_data):
    flat_data = eeg_data.flatten()
    if len(flat_data) > 209:
        return flat_data[:209]
    else:
        return np.pad(flat_data, (0, 209 - len(flat_data)), 'constant')


class SchizophreniaRiskApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Schizophrenia Risk Assessment App")
        self.root.attributes('-fullscreen', True)  # Fullscreen application

        # Initialize variables
        self.eeg_data = None
        self.model = None
        self.report_content = None
        self.risk_level = None
        self.explanation_text = None
        self.recommendations_text = None
        self.doctor_guidance_text = None

        # Load and set the background image
        try:
            image_path = r"C:\Users\tanis\Desktop\img.jpg"
            if not os.path.exists(image_path):
                raise FileNotFoundError(f"File not found: {image_path}")
                
            bg_image = Image.open(image_path)
            screen_width = self.root.winfo_screenwidth()
            screen_height = self.root.winfo_screenheight()
            bg_image = bg_image.resize((screen_width, screen_height), Image.Resampling.LANCZOS)
            
            self.bg = ImageTk.PhotoImage(bg_image)
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load background image: {e}")
            self.root.destroy()
            return

        # Display the background image
        self.bg_label = tk.Label(self.root, image=self.bg)  # Persistent reference
        self.bg_label.image = self.bg 
        self.bg_label.place(x=0, y=0, relwidth=1, relheight=1)

        # Add title label
        title_label = tk.Label(
            self.root,
            text="Schizophrenia Risk Assessment App",
            font=("Arial", 30, "bold"),
            bg="white",
            fg="darkblue",
            relief="ridge"
        )
        title_label.pack(pady=20)

        # Frame for main buttons
        button_frame = tk.Frame(self.root, bg="white")
        button_frame.pack(pady=40)

        # Upload EEG Data Button
        self.upload_btn = tk.Button(
            button_frame,
            text="Upload EEG Data",
            font=("Arial", 18),
            bg="lightblue",
            command=self.upload_eeg_data
        )
        self.upload_btn.grid(row=0, column=0, padx=30, pady=20)

        # Start Assessment Button (disabled initially)
        self.start_btn = tk.Button(
            button_frame,
            text="Start Risk Assessment",
            font=("Arial", 18),
            bg="lightgreen",
            state=tk.DISABLED,
            command=self.start_assessment
        )
        self.start_btn.grid(row=0, column=1, padx=30, pady=20)


        # Download Report Button
        self.download_btn = tk.Button(
            button_frame,
            text="Download Report",
            font=("Arial", 18),
            bg="lightpink",
            command=self.download_report
        )
        self.download_btn.grid(row=0, column=3, padx=30, pady=20)

        # Exit Button
        self.exit_btn = tk.Button(
            button_frame,
            text="Exit",
            font=("Arial", 18),
            bg="lightcoral",
            command=self.exit_app
        )
        self.exit_btn.grid(row=0, column=4, padx=30, pady=20)

        
        self.output_text = tk.Text(self.root, wrap=tk.WORD, font=("Arial", 14), bg="black", fg="white")
        self.output_text.pack(fill=tk.BOTH, expand=True, padx=300, pady=20)
        self.output_text.pack_forget() 

       

    def upload_eeg_data(self):
        # Implementation remains the same
         file_path = filedialog.askopenfilename(
            title="Select EEG File",
            filetypes=[("EDF Files", "*.edf"), ("CSV Files", "*.csv")]
        )

         if not file_path:
            return  # User canceled the file dialog

         try:
            # Load the file and preprocess it
            if file_path.endswith(".edf"):
                signals, _, _ = highlevel.read_edf(file_path)
                self.eeg_data = np.array(signals).T  # Transpose to match expected format
            elif file_path.endswith(".csv"):
                self.eeg_data = pd.read_csv(file_path).values
            else:
                raise ValueError("Unsupported file format. Please upload an EDF or CSV file.")

            # Verify the data shape
            if self.eeg_data.ndim != 2:
                raise ValueError(f"Unexpected data shape: {self.eeg_data.shape}. Expected a 2D array.")

            messagebox.showinfo("Upload Successful", "EEG data uploaded successfully.")
            self.start_btn.config(state=tk.NORMAL)  # Enable the Start Assessment button
             

         except Exception as e:
            messagebox.showerror("Error", f"Failed to process the file: {e}")


    def start_assessment(self):
        # Implementation remains the same
        if self.eeg_data is None:
            messagebox.showerror("Error", "No EEG data uploaded. Please upload a file first.")
            return

        try:
            # Extract features
            feature_vector = extract_features(self.eeg_data).reshape(1, -1)

            # Load the ML model
            try:
                self.model = joblib.load("svm_model.pkl")
            except Exception as e:
                raise ValueError("Failed to load the model. Ensure 'svm_model.pkl' is a valid joblib file.") from e

            # Ensure feature vector matches model input
            if feature_vector.shape[1] != self.model.n_features_in_:
                raise ValueError(
                    f"Mismatch in feature dimensions. Model expects {self.model.n_features_in_} features, "
                    f"but received {feature_vector.shape[1]}."
                )

            # Make predictions
            prediction = self.model.predict(feature_vector)
            self.risk_level = "High" if prediction[0] == 1 else "Low"

            if prediction[0] == 1:
                self.explanation_text = (
                    "The model detected patterns in the EEG data associated with high schizophrenia risk. "
                    "This could indicate abnormalities in brain activity, such as disrupted neural connectivity or atypical oscillatory patterns. "
                    "We recommend consulting a mental health professional immediately for a detailed evaluation and treatment plan."
                )
                self.recommendations_text = (
                    "1. Avoid stress and maintain a healthy lifestyle.\n"
                    "2. Ensure adequate sleep and regular physical activity.\n"
                    "3. Seek therapy or counseling sessions as needed.\n"
                    "4. Follow up with a psychiatrist for medication or advanced interventions."
                )
                self.doctor_guidance_text = (
                    "This report highlights potential abnormalities in the patient's EEG signals. "
                    "Doctors can use this data to identify specific neural patterns, compare with clinical observations, "
                    "and plan targeted interventions, such as cognitive therapies or medication adjustments."
                )
            else:
                self.explanation_text = (
                    "The model did not detect significant patterns in the EEG data associated with schizophrenia. "
                    "This indicates a low likelihood of schizophrenia-related brain activity at this time."
                )
                self.recommendations_text = (
                    "1. Maintain a healthy routine and stay stress-free.\n"
                    "2. Continue regular health checkups to monitor mental well-being.\n"
                    "3. Stay informed about early warning signs of mental health disorders.\n"
                    "4. Seek professional advice if unusual symptoms arise."
                )
                self.doctor_guidance_text = (
                    "The absence of significant abnormalities in the EEG data suggests no immediate schizophrenia risk. "
                    "Doctors may still monitor the patient for any emerging symptoms and use this report as a baseline for future assessments."
                )

            self.report_content = (
                f"Schizophrenia Risk Assessment Report\n\n"
                f"Risk Level: {self.risk_level}\n\n"
                f"Explanation:\n{self.explanation_text}\n\n"
                f"Recommendations:\n{self.recommendations_text}\n\n"
                f"Guidance for Doctors:\n{self.doctor_guidance_text}"
            )

            self.output_text.pack(fill=tk.BOTH, expand=True, padx=300, pady=20)
            self.output_text.insert(tk.END, self.report_content + "\n")

        except Exception as e:
            self.output_text.insert(tk.END, f"Error: {e}\n")

   
    def download_report(self):
        """Allow the user to download the report in a formatted PDF."""
        if not self.report_content:
            self.output_text.insert(tk.END, "No report to download.\n")
            return
    
        # Ask the user where to save the report
        file_path = filedialog.asksaveasfilename(defaultextension=".pdf",
                                                 filetypes=[("PDF Files", "*.pdf")])
        if not file_path:
            return
    
        try:
            from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, ListFlowable, ListItem
            from reportlab.lib.pagesizes import letter
            from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
            from reportlab.lib import colors
    
            # Get default styles and define custom styles
            styles = getSampleStyleSheet()
            styles.add(ParagraphStyle(name="CustomHeading", fontSize=18, textColor=colors.darkblue, spaceAfter=12))
            styles.add(ParagraphStyle(name="CustomSubheading", fontSize=14, textColor=colors.darkblue, spaceAfter=8))
            styles.add(ParagraphStyle(name="CustomBodyText", alignment=4, fontSize=12))  # Alignment 4 is justify
    
            # Prepare content for the report
            risk_level = f"Risk Level: {self.risk_level}"
            explanation = self.explanation_text
            recommendations = self.recommendations_text.split("\n")
            doctor_guidance = self.doctor_guidance_text.split("\n")
    
            # Create PDF elements
            elements = [
                Paragraph("Schizophrenia Risk Assessment Report", styles["CustomHeading"]),
                Spacer(1, 12),
                Paragraph(risk_level, styles["CustomBodyText"]),
                Spacer(1, 12),
                Paragraph("Explanation:", styles["CustomSubheading"]),
                Paragraph(explanation, styles["CustomBodyText"]),
                Spacer(1, 12),
                Paragraph("Recommendations:", styles["CustomSubheading"]),
                ListFlowable(
                    [ListItem(Paragraph(item.strip(), styles["CustomBodyText"])) for item in recommendations if item.strip()],
                    bulletType="bullet"
                ),
                Spacer(1, 12),
                Paragraph("Guidance for Doctors:", styles["CustomSubheading"]),
                ListFlowable(
                    [ListItem(Paragraph(item.strip(), styles["CustomBodyText"])) for item in doctor_guidance if item.strip()],
                    bulletType="bullet"
                ),
            ]
    
            # Create and save the PDF
            doc = SimpleDocTemplate(file_path, pagesize=letter)
            doc.build(elements)
            self.output_text.insert(tk.END, f"Report successfully saved to {file_path}.\n")
        except ImportError as ie:
            self.output_text.insert(tk.END, f"Missing library: {ie}. Ensure you have installed the 'reportlab' module.\n")
        except PermissionError:
            self.output_text.insert(tk.END, "Permission denied: Please check if the file is open or try another location.\n")
        except Exception as e:
            self.output_text.insert(tk.END, f"An unexpected error occurred: {e}\n")



    def exit_app(self):
        # Implementation remains the same
        if messagebox.askyesno("Exit", "Are you sure you want to exit?"):
            self.root.destroy()


if __name__ == "__main__":
    root = tk.Tk()
    app = SchizophreniaRiskApp(root)
    root.mainloop()
